<a href="https://colab.research.google.com/github/dpcks/DACON--AI-/blob/main/pre_NV_X_ipynb_(84688).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer

import torch

from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/딥러닝 프로젝트/data/"
SEED = 88

In [ ]:
if not os.path.isdir("image"):
    !cp "{DATA_PATH}open.zip" "open.zip"
    !unzip -qq "open.zip"

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [ ]:
test

,id,img_path,overview
0,TEST_00000,./image/test/TEST_00000.jpg,신선한 재료로 정성을 다해 만들었다. 늘 변함없는 맛과 서비스로 모실것을 약속한다.
1,TEST_00001,./image/test/TEST_00001.jpg,"청청한 해역 등량만과 율포해수욕장이 한눈에 내려다 보이는 위치에 있으며, 막 잡은 ..."
2,TEST_00002,./image/test/TEST_00002.jpg,장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방...
3,TEST_00003,./image/test/TEST_00003.jpg,다양한 형태의 청소년수련활동을 제공함으로써 청소년들이 민주사회의 주역이 될 수 있도...
4,TEST_00004,./image/test/TEST_00004.jpg,팔공산은 경산시의 북쪽에 위치한 해발 1192.3 m의 높은 산으로 신라시대에는 중...
...,...,...,...
7275,TEST_07275,./image/test/TEST_07275.jpg,"막국수와 수육을 주메뉴로 하며, 넓은 주차장이 마련되어 있어 주차하기 편리하다.<br>"
7276,TEST_07276,./image/test/TEST_07276.jpg,"통진두레문화센터는 우리고유의 전통무형문화와 민속예술을 계승/발전 시키고, 다양한 문..."
7277,TEST_07277,./image/test/TEST_07277.jpg,"수도권에서 가까운 위치, 문산천을 따라 걷는 산책코스, 한여름 더위를 날려버릴 시원..."
7278,TEST_07278,./image/test/TEST_07278.jpg,\n전남 구례군 관산리에 위치한 노고단 게스트하우스&호텔은 지리산을 파노라마로 관망...


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16986 entries, 0 to 16985
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        16986 non-null  object
 1   img_path  16986 non-null  object
 2   overview  16986 non-null  object
 3   cat1      16986 non-null  object
 4   cat2      16986 non-null  object
 5   cat3      16986 non-null  object
dtypes: object(6)
memory usage: 796.3+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7280 entries, 0 to 7279
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7280 non-null   object
 1   img_path  7280 non-null   object
 2   overview  7280 non-null   object
dtypes: object(3)
memory usage: 170.8+ KB


In [ ]:
train.nunique()

id          16986
img_path    16986
overview    16973
cat1            6
cat2           18
cat3          128
dtype: int64

In [ ]:
test.nunique()

id          7280
img_path    7280
overview    7280
dtype: int64

In [ ]:
le = LabelEncoder()
train["target"] = le.fit_transform(train['cat3'].to_numpy())
train

,id,img_path,overview,cat1,cat2,cat3,target
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,118
...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,118
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,31
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,31
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",73


In [ ]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
     |████████████████████████████████| 30.5 MB 1.2 MB/s 
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.14.0-py3-none-any.whl size=30560039 sha256=7028d5f3fd93ab7a9901d238073c766140fea30424a781ea5bc540d458fc1786
  Stored in directory: /root/.cache/pip/wheels/97/36/50/1fc99e4a3ce636ed30739cc2cb333b36e6e3778f365f1adf77
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()

In [ ]:
# lst = train["overview"].tolist()
# gen = kiwi.analyze(lst)
# train_overview = []

# for tokens in tqdm(gen,total=len(lst)):
#     token = [ token.form for token in tokens[0][0]]
#     train_overview.append(" ".join(token))

# lst = test["overview"].tolist()
# gen = kiwi.analyze(lst)
# test_overview = []
# for tokens in tqdm(gen,total=len(lst)):
#     token = [ token.form for token in tokens[0][0]]
#     test_overview.append(" ".join(token))

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 66.9 MB/s 
     |████████████████████████████████| 163 kB 63.9 MB/s 


In [ ]:
model_name = "kykim/bert-kor-base"

In [ ]:
from transformers import BertTokenizerFast, BertModel

tokenizer_bert = BertTokenizerFast.from_pretrained(model_name)
model_bert = BertModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_overview = np.array(train["overview"])
test_overview = np.array(test["overview"])
target = train.target.to_numpy()

In [ ]:
token = tokenizer_bert(train_overview[0], add_special_tokens=True,padding="max_length", truncation=True)
token

{'input_ids': [2, 5035, 8102, 8200, 8078, 26959, 4971, 13969, 18416, 8010, 17684, 23379, 8010, 24809, 17153, 8018, 2482, 28747, 8051, 14261, 14617, 19576, 8051, 18462, 14754, 15721, 20158, 22236, 8144, 28602, 2016, 7670, 18874, 24106, 28747, 8020, 14022, 4971, 16065, 14086, 23448, 8112, 14686, 14331, 25115, 3685, 8100, 19345, 14754, 5929, 8576, 3463, 4971, 8107, 6035, 23097, 37597, 31056, 14624, 30006, 13979, 22713, 2016, 14254, 2014, 4971, 13994, 2103, 8118, 8088, 22122, 2014, 29151, 29716, 14850, 4964, 15247, 23685, 34607, 8014, 14754, 2014, 17922, 6035, 8063, 7557, 8082, 8031, 7557, 25586, 33758, 13984, 2016, 2030, 20958, 2032, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self ,tokenizer , x, y = None ): 
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, idx): 
        item = {}
        item["x"] = self.__tokenizer(self.x[idx]) 
        if self.y is not None:
            item["y"] = torch.tensor(self.y[idx])
        return item
    def __tokenizer(self,text):
        inputs = self.tokenizer(text, add_special_tokens=True,padding="max_length", truncation=True)
        for k, v in inputs.items(): 
            inputs[k] = torch.LongTensor(v) 
        return inputs

In [ ]:
dt = MyDataset(tokenizer_bert,train_overview,target)
dl = torch.utils.data.DataLoader(dt, batch_size=2,shuffle=False) 
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2,  5035,  8102,  ...,     0,     0,     0],
         [    2, 16403, 22624,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}, 'y': tensor([120,   8])}

In [ ]:
# model = AutoModel.from_pretrained(model_name)

In [ ]:
batch["x"].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
outputs = model_bert(**batch["x"])
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
model_bert.config.hidden_size

768

In [ ]:
BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, model_name): 
        super().__init__()
        self.model = BertModel.from_pretrained(model_name)
        self.output_layer = torch.nn.Linear(self.model.config.hidden_size, 128)
    def forward(self, x):
        x = self.model(**x) 
        x = self.output_layer(x[0][:,0])
        return x

In [ ]:
model = Net(model_name)
model(batch["x"])

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 4.6747e-01,  6.4135e-01,  1.2310e+00,  1.3141e+00,  5.3520e-01,
         -2.2162e-01,  6.9604e-01, -4.9123e-01,  1.5506e-01, -6.2764e-01,
          4.7620e-01,  3.4269e-02, -8.5239e-01, -3.9395e-01,  1.7240e-01,
          2.8069e-01, -1.9565e-01,  1.2160e+00, -4.6294e-01, -2.9293e-01,
         -2.9697e-01, -9.6117e-02, -4.6796e-01, -7.9285e-01,  1.1798e+00,
          7.5994e-01,  3.9019e-01, -2.6766e-01,  1.3399e-01, -7.4305e-01,
          2.2859e-01, -1.7803e-02, -6.5832e-01, -7.9824e-01, -1.0297e-03,
          2.4791e-01, -5.9795e-01, -9.0884e-02,  2.3727e-01,  4.7627e-01,
          2.8861e-01,  4.6367e-01, -3.5681e-01,  4.5023e-01, -3.5458e-01,
          6.5186e-01, -3.9429e-02, -1.9760e-01,  7.7632e-02,  8.8438e-02,
         -6.7427e-01, -4.3038e-01, -7.4799e-02,  4.6339e-01, -4.3494e-02,
          1.6917e-01, -4.2039e-01,  4.4901e-01, -6.5858e-02, -5.3547e-01,
         -8.0267e-01,  4.8085e-01, -7.6954e-01, -4.7305e-03, -4.8427e-01,
          3.7893e-02, -5.4576e-01, -1.

In [ ]:
train.target.nunique()

128

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 
    model.train() 
    for batch in tqdm(dataloader): 
        pred = model(batch["x"].to(device)) 
        loss = loss_fn(pred, batch["y"].to(device)) 
        
        optimizer.zero_grad() 
        loss.backward()  
        optimizer.step() 
        
        epoch_loss += loss.item() 

    epoch_loss /= len(dataloader) 

    return epoch_loss 

In [ ]:
@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 

    
    pred_list = []
    softmax = torch.nn.Softmax(dim=1) 

    for batch in tqdm(dataloader):
        
        pred = model(batch["x"].to(device))
        if batch.get("y") is not None: 
            loss = loss_fn(pred, batch["y"].to(device))
            epoch_loss += loss.item()
        
        pred = softmax(pred)
        pred = pred.to("cpu").numpy() 
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list) 
    return epoch_loss , pred 

In [ ]:
DATA_PATH2 = "/content/drive/MyDrive/딥러닝 프로젝트/model_weight/"

In [ ]:
n_splits = 5
batch_size = 4
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류 손실객체

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
# is_holdout = False
# seed_everything(SEED)
# best_score_list = []
# for i,(tri,vai) in enumerate(cv.split(train_overview)):
#     if i != 2:
#         continue
#     model = Net(model_name).to(device)
#     optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)

#     train_dt = MyDataset(tokenizer_bert,train_overview[tri],target[tri])
#     valid_dt = MyDataset(tokenizer_bert,train_overview[vai],target[vai])
#     train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
#     valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False)

#     best_score = 0
#     patience = 0

#     for epoch in range(epochs):
        
#         train_loss = train_loop(train_dl, model, loss_fn,optimizer,device )
#         valid_loss , pred = test_loop(valid_dl, model, loss_fn,device  )
        
#         pred = np.argmax(pred, axis=1) 
#         true = target[vai] 
#         score = f1_score(true, pred , average="weighted")
#         print(train_loss,valid_loss,score)
#         patience += 1
#         if best_score < score:
#             patience = 0
#             best_score = score
#             torch.save(model.state_dict(),f"{DATA_PATH2}kykim_model_X_{i}.pth")

#         if patience == 2:
#             break
#     print(f"Fold ({i}), BEST F1: {best_score}")
#     best_score_list.append(best_score)

#     torch.cuda.empty_cache()

#     if is_holdout:
#         break

In [ ]:
# np.mean(best_score_list)

In [ ]:
is_holdout = False

In [ ]:
test_dt = MyDataset(tokenizer_bert,test_overview)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=batch_size, shuffle=False)
pred_list = []
for i in range(n_splits):
    model = Net(model_name).to(device)
    state_dict = torch.load(f'/content/drive/MyDrive/딥러닝 프로젝트/best model_weight/kykim_model_X_{i}.pth',f'/content/drive/MyDrive/딥러닝 프로젝트/model_weight/NV_X_aw_{i}.pth')
    model.load_state_dict(state_dict)
    _ , pred = test_loop(test_dl, model, loss_fn,device  )
    pred_list.append(pred)
    if is_holdout:
        break

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: ignored

In [ ]:
np.save(f'{DATA_PATH2}pred_list(pre_mean_ensemble)best_channy.npy',pred_list)

In [ ]:
pred=np.mean(pred_list,axis=0)
np.save(f'{DATA_PATH2}pred_best_channy(post_mean_ensemble).npy',pred)

In [ ]:
pred = np.mean(pred_list,axis=0)
pred = np.argmax(pred, axis=1)
pred
np.save(f'{DATA_PATH2}pred_best_channy(post_mean_ensemble).npy',pred)

# 앙상블

In [ ]:
# channy_pred = np.load("/content/drive/MyDrive/딥러닝 프로젝트/data/pred_list(pre_mean_ensemble)best_channy.npy")
# moa_pred = np.load("/content/drive/MyDrive/딥러닝 프로젝트/data/mollang_pred_lst(pre_mean_ensemble).npy")


In [ ]:
# channy_pred.shape,moa_pred.shape

In [ ]:
# c_pred = np.mean(channy_pred,axis=0)
# m_pred = np.mean(moa_pred, axis=0)

# pred = np.mean([c_pred,m_pred],axis=0)

In [ ]:
# pred = np.argmax(pred, axis=1)

In [ ]:
submit = pd.read_csv("sample_submission.csv")
submit["cat3"] = le.inverse_transform(pred)
submit

In [ ]:
DATA_PATH3 = "/content/drive/MyDrive/딥러닝 프로젝트/summit/"

In [ ]:
submit.to_csv(f'{DATA_PATH3}앙상블.csv', index=False)